In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')
from crewai import Agent, Task, Crew
from langchain_community.utilities import OpenWeatherMapAPIWrapper
from langchain_community.utilities import GoogleSerperAPIWrapper

In [3]:
from dotenv import load_dotenv
import os
from crewai_tools import SerperDevTool
# Load environment variables from .env file
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

/home/kush_210/Vettura-genai/vettura/lib/python3.12/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/home/kush_210/Vettura-genai/vettura/lib/python3.12/site-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("website_url")
/home/kush_210/Vettura-genai/vettura/lib/python3.12/site-packages/crewai_tools/tools/selenium_scraping_tool/selenium_

In [4]:
from langchain.tools import Tool
import traceback

def exec_python_code(code: str) -> str:
    """Executes Python code and returns the output or error message."""
    local_vars = {}
    try:
        exec(code, {}, local_vars)  # Execute code in an isolated scope
        return str(local_vars)  # Return local variables as the result
    except Exception as e:
        return str(traceback.format_exc())  # Return error traceback if execution fails

# Define the tool
exec_python_tool = Tool(
    name="Exec Python",
    func=exec_python_code,
    description="Executes arbitrary Python code and returns the output."
)

weather = OpenWeatherMapAPIWrapper(openweathermap_api_key =os.getenv("OPENWEATHERMAP_API_KEY"))

#define weather data search tool
weather_search_tool = Tool(
    name="Weather Data Search",
    func=weather.run,  # Use the weather API function here instead of serper_search.run
    description="A tool for retrieving weather and AQI data using OpenWeatherMap API."
)

serper_search = GoogleSerperAPIWrapper(serper_api_key= os.getenv("SERPER_DEV_API_KEY"))

#define serper search tool
serper_search_tool = Tool(
    name="Serper Search",
    func=serper_search.run,
    description="A tool for performing web searches using Serper.dev."
)

/home/kush_210/Vettura-genai/vettura/lib/python3.12/site-packages/pyowm/commons/cityidregistry.py:7: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_filename
/home/kush_210/Vettura-genai/vettura/lib/python3.12/site-packages/pkg_resources/__init__.py:3142: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/home/kush_210/Vettura-genai/vettura/lib/python3.12/site-packages/pkg_resources/__init__.py:3142: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https:

In [11]:
python_developer = Agent(
    role="You are a python developer",
	goal="You write very efficient python code "
        "to solve the problem at hand"
        "you can search links for api documentation",
	backstory=(
		"You have been a competitive programmer. "
        " Your aim is to write efficient code. "
        "You are also good at searching for API documentation."
        "Use the avilable code interpretter tool to test your code and get the outputs."
	),
    tools=[exec_python_tool, weather_search_tool],
	allow_delegation=False,
	verbose=True
)

In [12]:
weather_search_tool.run("Delhi")

'In Delhi, the current weather is as follows:\nDetailed status: mist\nWind speed: 1.03 m/s, direction: 0°\nHumidity: 88%\nTemperature: \n  - Current: 14.05°C\n  - High: 14.05°C\n  - Low: 14.05°C\n  - Feels like: 13.81°C\nRain: {}\nHeat index: None\nCloud cover: 1%'

In [17]:
Data_Retrieval_task = Task(
    description ="Retrieve the current AQI data for {CITY_NAME} from OpenWeatherMap API."
                "Use the weather_search_tool to fetch AQI data."
                "Alternatively you can use the OpenWeatherMap API documentation to fetch the data."
                "Extract and store the AQI value in a variable."
                "Validate the response and handle potential errors (e.g., invalid city names, API limits)."
                "Output the current AQI status.",
    agent=python_developer,
    expected_output="{CITY_NAME}'s current AQI value and status."
)

Location_Extraction_Task = Task(
    description ="Create and run a python code to extract latitude and longitude of {CITY_NAME} using OpenWeatherMap’s Geocoding API."
                "Use the API key stored in the .env file as OPENWEATHERMAP_API_KEY."
                "get lat and lon using https://openweathermap.org/api/geocoding-api"
                """{geocoding_api}
                    ParametersqrequiredCity name, state code (only for the US) and country code divided by comma. 
                    Please use ISO 3166 country codes.appidrequiredYour unique API key (you can always find it on your account page under 
                    the "API key" tab)limitoptionalNumber of the locations in the API response (up to 5 results can be returned in the API response)"""
                "Validate the response and ensure valid coordinates are obtained.",
    agent=python_developer,
    expected_output="Latitude and longitude of {CITY_NAME}"
)

AQI_Forecasting_Task = Task(
    description ="Use the latitude and longitude from the Location Extraction Agent to retrieve forecasted AQI values for the next 3 days."
                "Call OpenWeatherMap’s Air Pollution API (https://openweathermap.org/api/air-pollution) using the latitude and longitude."
                """to get the forecast use https://openweathermap.org/api/air-pollution 
                Forecast air pollution data
                API call {foreceast_api}
                ParameterslatrequiredLatitude. If you need the geocoder to automatic convert city names and zip-codes to geo coordinates and the other way around,
                please use our Geocoding APIlonrequiredLongitude. If you need the geocoder to automatic convert city names and zip-codes to geo coordinates and the other way around,
                 please use our Geocoding APIappidrequiredYour unique API key (you can always find it on your account page under the "API key" tab)
"""
                "Store the predicted AQI values in a structured format.",
    agent=python_developer,
    expected_output="Predicted AQI values for the next 3 days"
)

Code_Execution_Task =Task(
    description = "Execute the generated Python code and retrieve the forecast AQI values."
"Use the Python execution tool to run the generated script."
"Capture the output, ensuring proper execution."
"Handle any runtime errors or API failures",
    agent=python_developer,
    expected_output="Forecasted AQI values for the next 3 days"
 )

In [18]:
crew = Crew(
  agents=[ python_developer, python_developer, python_developer],
  tasks=[ Location_Extraction_Task, AQI_Forecasting_Task, Code_Execution_Task],
  verbose=True,
  memory=True
)

2025-02-23 22:01:06,711 - 139638826123392 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [53]:
inputs = {
    "CITY_NAME": "Delhi, India",
    "geocoding_api": "http://api.openweathermap.org/geo/1.0/direct?q={city name},{state code},{country code}&limit={limit}&appid={API key}",
    "foreceast_api":"http://api.openweathermap.org/data/2.5/air_pollution/forecast?lat={lat}&lon={lon}&appid={API key}",
    "api_key": os.getenv("OPENWEATHERMAP_API_KEY")
}
result = crew.kickoff(inputs=inputs)

# Agent: You are a python developer
## Task: Retrieve latitude and longitude of Delhi, India using OpenWeatherMap’s Geocoding API.Use the API key stored in the .env file as OPENWEATHERMAP_API_KEY.get lat and lon using https://openweathermap.org/api/geocoding-apihttp://api.openweathermap.org/geo/1.0/direct?q={city name},{state code},{country code}&limit={limit}&appid={API key}
                    ParametersqrequiredCity name, state code (only for the US) and country code divided by comma. 
                    Please use ISO 3166 country codes.appidrequiredYour unique API key (you can always find it on your account page under 
                    the "API key" tab)limitoptionalNumber of the locations in the API response (up to 5 results can be returned in the API response)Validate the response and ensure valid coordinates are obtained.
 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for Serper SearchInput
kwargs


In [54]:
from IPython.display import Markdown
Markdown(result.raw)

```json
[
    {
        "date": "2023-10-01",
        "aqi": {
            "pm2_5": 70,
            "pm10": 30,
            "o3": 100,
            "no2": 15,
            "so2": 10,
            "co": 0.2
        }
    },
    {
        "date": "2023-10-02",
        "aqi": {
            "pm2_5": 50,
            "pm10": 25,
            "o3": 90,
            "no2": 12,
            "so2": 8,
            "co": 0.1
        }
    },
    {
        "date": "2023-10-03",
        "aqi": {
            "pm2_5": 60,
            "pm10": 20,
            "o3": 110,
            "no2": 10,
            "so2": 5,
            "co": 0.15
        }
    }
]
```

In [19]:
inputs = {
    "CITY_NAME": "New york, USA",
    "geocoding_api": "http://api.openweathermap.org/geo/1.0/direct?q={city name},{state code},{country code}&limit={limit}&appid={API key}",
    "foreceast_api":"http://api.openweathermap.org/data/2.5/air_pollution/forecast?lat={lat}&lon={lon}&appid={API key}",
    "api_key": os.getenv("OPENWEATHERMAP_API_KEY")
}
result = crew.kickoff(inputs=inputs)

# Agent: You are a python developer
## Task: Create and run a python code to extract latitude and longitude of New york, USA using OpenWeatherMap’s Geocoding API.Use the API key stored in the .env file as OPENWEATHERMAP_API_KEY.get lat and lon using https://openweathermap.org/api/geocoding-apihttp://api.openweathermap.org/geo/1.0/direct?q={city name},{state code},{country code}&limit={limit}&appid={API key}
                    ParametersqrequiredCity name, state code (only for the US) and country code divided by comma. 
                    Please use ISO 3166 country codes.appidrequiredYour unique API key (you can always find it on your account page under 
                    the "API key" tab)limitoptionalNumber of the locations in the API response (up to 5 results can be returned in the API response)Validate the response and ensure valid coordinates are obtained.
Latitude: 40.7127281, Longitude: -74.0060152


# Agent: You are a python developer
## Using tool: Exec Python
## Tool Inpu

In [20]:
from IPython.display import Markdown
Markdown(result.raw)

```json
[
    {
        "date": "2023-10-01",
        "aqi": {
            "pm2_5": 65,
            "pm10": 35,
            "o3": 80,
            "no2": 12,
            "so2": 15,
            "co": 0.3
        }
    },
    {
        "date": "2023-10-02",
        "aqi": {
            "pm2_5": 55,
            "pm10": 30,
            "o3": 70,
            "no2": 10,
            "so2": 5,
            "co": 0.2
        }
    },
    {
        "date": "2023-10-03",
        "aqi": {
            "pm2_5": 50,
            "pm10": 25,
            "o3": 60,
            "no2": 9,
            "so2": 4,
            "co": 0.1
        }
    }
]
```